In [1]:
import cv2
import numpy as np



In [2]:
# kamera aç
cap = cv2.VideoCapture(0)

In [3]:
# bir tane frame oku
ret, frame = cap.read()

In [4]:
if ret == False:
    print("Uyarı")

In [5]:
#detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
face_rects = face_cascade.detectMultiScale(frame)

(face_x, face_y, w, h) = tuple(face_rects[0])
track_window = (face_x, face_y, w, h) #meanshift algoritma inputu

In [6]:
#region of interest
roi = frame[face_y:face_y+h,face_x:face_x+w] # roi = face

In [7]:
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

In [8]:
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180]) #takip için histogram
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)


array([[2.5499998e+02],
       [4.0497856e+01],
       [7.1622055e+01],
       [7.3988220e+01],
       [7.4989288e+01],
       [9.4100639e+01],
       [1.0729657e+02],
       [9.0733398e+01],
       [1.0320128e+02],
       [7.9994644e+01],
       [6.1520340e+01],
       [5.0235542e+01],
       [3.7494644e+01],
       [3.8586723e+01],
       [2.5663811e+01],
       [2.1659529e+01],
       [1.3650963e+01],
       [9.6466808e+00],
       [8.7366161e+00],
       [5.9154172e+00],
       [9.6466808e+00],
       [4.0952888e+00],
       [1.2740899e+00],
       [6.1884365e+00],
       [4.0952888e+00],
       [4.3683081e+00],
       [3.0032120e+00],
       [1.5471091e+00],
       [2.8211989e+00],
       [0.0000000e+00],
       [7.1440041e+01],
       [0.0000000e+00],
       [0.0000000e+00],
       [6.3704497e-01],
       [2.7301925e-01],
       [7.2805136e-01],
       [2.6391861e+00],
       [0.0000000e+00],
       [2.1841540e+00],
       [0.0000000e+00],
       [2.0931478e+00],
       [0.000000

In [9]:
#takip için gerekli durdurma kriterleri
#count = hesaplanacak maksimum öge sayısı
#eps = değişiklik

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1)

while True:
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        #histogramı bir görüntüde bulmak için kullanıyoruz
        #piksel karşılaştırma
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        ret,track_window = cv2.meanShift(dst, track_window, term_crit)
        
        x,y,w,h = track_window
        
        img2 = cv2.rectangle(frame,(x,y),(x+w, y+h),(0,0,255),5)
        
        cv2.imshow("Takip",img2)
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
cap.release()
cv2.destroyAllWindows()